In [1]:
import itertools
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.io as pio
pio.renderers.default = 'iframe'

pd.options.display.max_rows = 100

In [2]:
df = pd.read_json('../../data/fc2b/fc2b.json')
df.sample(10)

,blog,entry,genre_id,genre_name,subgenre_id,subgenre_name,text,title,toxicity
12073,kaguratokyo.blog9.fc2.com,blog-entry-5393.html,13,アイドル・芸能,123,その他,丹下桜さんが Voidoll と Bugdoll 役で出演されているゲーム「 #コンパス」の...,桜色の日々 アニメ「 #コンパス2ꓸ0 」放送開始月決定！,0
7542,ken0314.blog.fc2.com,blog-entry-2888.html,41,心と身体,314,メンタルヘルス,「学校ほど閉鎖的な組織はない」 いじめ自殺「代理人」が挑む闘い方編集委員・石橋英昭•\t• ...,「学校ほど「学校ほど閉鎖的な組織はない」 いじめ自殺「代理人」が挑む闘い方 - 田中幸子のひ...,0
7060,gharrierjp.blog.fc2.com,blog-entry-4051.html,22,ブログ,19,写真ブログ,【悲報】指定暴力団「稼いだ金はポケモンカードに変えてマネロン。海外に持ち出して現地で現金化す...,２４／１２／１７の日記～リアルにでかい金が動く二度と経験できない懐かしい過去｜豪雪地帯酒店・...,0
11798,ginmaku3.blog25.fc2.com,blog-entry-3757.html,30,映画,46,ファンタジー,古今東西の映画・洋画・邦画・ドラマのカスタムＤＶＤ、Blu-rayラベルを公開しています！,GINMAKU Custom DVD＆Blu-ray labels blog版／映画・洋画・...,0
9977,ponko69.blog118.fc2.com,blog-entry-7174.html,9,政治・経済,55,政治活動,＊Comment\n\n\n★泥棒が人を泥棒呼ばわりするニチホ\n「政策がこのように間違って...,反日勢力を斬る 百・有、文春のインタビューに切れるの愚,0
2729,youkaitouzai.blog97.fc2.com,blog-entry-5906.html,6,ニュース,355,時事,▼12.13 フランス、あらたにフランソワ・バイルを総理大臣に任命。就任演説では「目の前に高...,妖界東西新聞 まる煮え入る,0
6146,tulip18.blog75.fc2.com,blog-entry-657.html,41,心と身体,312,病気・症状,毎月通院している内科へ行って、年に1度の検査結果を聞いてきた。何も問題なく、数値も良かった☆...,重い食アレと喘息・小児リウマチ予備軍の娘のおはなし インフルが流行りだしたぞぉぉ～,0
5957,shokoty.blog121.fc2.com,blog-entry-5026.html,41,心と身体,315,癒し,今年最後の満月の夜から ２日が経ち...２階にいた夫が 「おーい 月が出たで」とそんな事 滅...,ハワイ発 まぁるい氣を貴方に 寒月 (Cold Moon),0
10880,katsuraomura.blog.fc2.com,blog-entry-1007.html,39,福祉・ボランティア,276,ボランティア,こんにちわ。「かつらお動物見守り隊」です。 今回は、訃報です。 実はもう３ケ月前のことだった...,かつらお動物見守り隊 - 福島の被災犬、空くん（旧名ボウボウ）、亡くなりました。,0
12927,blauekatze777.blog.fc2.com,blog-entry-401.html,10,ゲーム,71,RPG,おはようございますっ とりあえず丼の方でも告知したのでこちらでも！ここの所体調崩して寝てたり...,12/22(日)ルティエフリマ＆オークション情報 追加っ - RO B鯖フリマ,0


In [3]:
[df[c].unique() for c in ['genre_name', 'toxicity']]

[array(['ファッション・ブランド', '独身・フリー', 'オンラインゲーム', '小説・文学', 'アダルト', 'ギャンブル',
        '恋愛', '本・雑誌', '車・バイク', '映画', 'サブカル', '政治・経済', '趣味・実用', '音楽', 'ペット',
        '育児', '日記', 'グルメ', '謎', 'ニュース', '携帯電話・PHS', '地域情報', 'コンピュータ',
        '学問・文化・芸術', '福祉・ボランティア', '学校・教育', '海外情報', 'お笑い', '写真', 'スポーツ',
        'ファイナンス', 'ビジネス', 'ヘルス・ダイエット', 'ライフ', '株式・投資・マネー', 'ブログ',
        'アフィリエイト', '結婚・家庭生活', 'ゲーム', '就職・お仕事', 'テレビ・ラジオ', 'アイドル・芸能',
        'アニメ・コミック', '心と身体', 'モブログ', 'その他', '旅行'], dtype=object),
 array([0, 1])]

In [4]:
# カテゴリ (y) について、均等に分割する関数
def train_test_split(df, columns, test_size=0.1, val_size=None):
    def qval(c, v):
        if types[c] in (object, ):
            return f'"{v}"'
        else:
            return v;

    types = df[columns].dtypes
    value_sets = [df[c].unique() for c in columns]

    prod = [p for p in itertools.product(*value_sets)]

    all_train_indices, all_test_indices, all_val_indices = [], [], []

    for p in prod:
        q = ' & '.join([f'{c[0]} == {qval(c[0], c[1])}' for c in zip(columns, p)])
        data = df.query(q)
        indices = data.index.values
        np.random.shuffle(indices)
        
        n_test = int(len(indices) * test_size)
        test_indices = indices[:n_test]
        train_indices = indices[n_test:]

        if val_size:
            n_val = int(len(indices) * val_size)
            val_indices = train_indices[:n_val]
            train_indices = train_indices[n_val:]
            all_val_indices.extend(val_indices)

        all_train_indices.extend(train_indices)
        all_test_indices.extend(test_indices)

    np.random.shuffle(all_train_indices)
    np.random.shuffle(all_test_indices)
    
    if val_size:
        np.random.shuffle(all_val_indices)
        return all_train_indices, all_test_indices, all_val_indices
    else:
        return all_train_indices, all_test_indices


def plot(train_indices, test_indices):
    fig = go.Figure()
    fig.add_trace(go.Histogram(name='train', x=df.iloc[train_indices]["genre_name"].values))
    fig.add_trace(go.Histogram(name='test', x=df.iloc[test_indices]["genre_name"].values))
    fig.update_layout(barmode='group')
    fig.update_traces(opacity=0.5)
    fig.update_xaxes(categoryorder="total descending")
    fig.show()

train_indices, test_indices = train_test_split(df, columns=['genre_name'], test_size=0.25)

plot(train_indices, test_indices)

In [5]:
# あとから知った別解 (stratify)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.index.values,
                                                    df.genre_id,
                                                    test_size=0.25,
                                                    random_state=42,
                                                    stratify=df.genre_id)

plot(train_indices, test_indices)

In [6]:
def categorical_kfold(df, columns, K=3, val_size=None):
    def qval(c, v):
        if types[c] in (object, ):
            return f'"{v}"'
        else:
            return v;

    types = df[columns].dtypes
    value_sets = [df[c].unique() for c in columns]

    prod = [p for p in itertools.product(*value_sets)]

    folds = [[] for _ in range(K)]
    if val_size:
        val_folds = [[] for _ in range(K)]

    for p in prod:
        q = ' & '.join([f'{c[0]} == {qval(c[0], c[1])}' for c in zip(columns, p)])
        data = df.query(q)
        indices = data.index.values
        np.random.shuffle(indices)
        
        for i, split in enumerate(np.array_split(indices, K)):
            if val_size:
                val_len = int(len(split) * val_size)
                folds[i].append(split[val_len:])
                val_folds[i].append(split[:val_len])
            else:
                folds[i].append(split)

    folds = [np.hstack(fold) for fold in folds]
    if val_size:
        val_folds = [np.hstack(fold) for fold in val_folds]        

    for n in range(K):
        train_indices = np.hstack([indices for i, indices in enumerate(folds) if i != n])
        if val_size:
            val_indices = np.hstack([indices for i, indices in enumerate(val_folds) if i != n])
            test_indices = np.r_[folds[n], val_folds[n]]
            yield train_indices, val_indices, test_indices
        else:
            test_indices = folds[n]
            yield train_indices, test_indices

data = [(train_indices, test_indices) for (train_indices, test_indices) in categorical_kfold(df, columns=['genre_name', 'toxicity'], K=3)]

def plot_kfold(data):
    fig = make_subplots(rows=len(data), cols=1, shared_xaxes=True, vertical_spacing = 0.03)
    for i, (train_indices, test_indices) in enumerate(data):
        fig.add_trace(go.Histogram(name='train', x=df.iloc[train_indices]["genre_name"].values, marker_color="blue"), row=i+1, col=1)
        fig.add_trace(go.Histogram(name='test', x=df.iloc[test_indices]["genre_name"].values, marker_color="red"), row=i+1, col=1)
    fig.update_layout(height=1000, width=1000, barmode='group', margin=dict(t=20, b=0, l=10, r=10), showlegend=False)
    fig.update_traces(opacity=0.5)
    fig.show()

In [7]:
plot_kfold(data)

In [8]:
# あとから知った別解
from sklearn.model_selection import StratifiedKFold

sk = StratifiedKFold(n_splits=3)
data = [(train_indices, test_indices) for (train_indices, test_indices) in sk.split(df.index, df.genre_id)]

/root/.cache/pypoetry/virtualenvs/datasets-xcsZ-3kG-py3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning:

The least populated class in y has only 2 members, which is less than n_splits=3.



In [9]:
plot_kfold(data)

In [10]:
data = [(train_indices, val_indices, test_indices)
        for (train_indices, val_indices, test_indices)
        in categorical_kfold(df, columns=['genre_name', 'toxicity'], K=3, val_size=0.25)]

def plot_kfold(data):
    fig = make_subplots(rows=len(data), cols=1, shared_xaxes=True, vertical_spacing = 0.03)
    for i, (train_indices, val_indices, test_indices) in enumerate(data):
        fig.add_trace(go.Histogram(name='train', x=df.iloc[train_indices]["genre_name"].values, marker_color="blue"), row=i+1, col=1)
        fig.add_trace(go.Histogram(name='validate', x=df.iloc[val_indices]["genre_name"].values, marker_color="red"), row=i+1, col=1)
        fig.add_trace(go.Histogram(name='test', x=df.iloc[test_indices]["genre_name"].values, marker_color="green"), row=i+1, col=1)
    fig.update_layout(height=1000, width=1000, barmode='group', margin=dict(t=20, b=0, l=10, r=10), showlegend=False)
    fig.update_traces(opacity=0.5)
    fig.show()

def plot(train_indices, val_indices, test_indices):
    fig = go.Figure()
    fig.add_trace(go.Histogram(name='train', x=df.iloc[train_indices]["genre_name"].values))
    fig.add_trace(go.Histogram(name='validate', x=df.iloc[val_indices]["genre_name"].values))
    fig.add_trace(go.Histogram(name='test', x=df.iloc[test_indices]["genre_name"].values))
    fig.update_layout(barmode='group')
    fig.update_traces(opacity=0.5)
    fig.update_xaxes(categoryorder="total descending")
    fig.show()

plot_kfold(data)

In [11]:
plot(data[0][0], data[0][1], data[0][2])